In this notebook we plot some of the cycles from the subsets from data_COIL20 generated by the "COIL20_class.py" file

In [ ]:
from PIL import Image
import numpy as np
import os

import IBloFunMatch_inter as ibfm

import matplotlib.pyplot as plt
import networkx as nx
from scipy.spatial import distance

In [ ]:
CLASS_LIST = list(range(4))
NUM_SAMPLES = 72
NUM_EXP = 20

In [ ]:
# Create list of labels
y=[]
for c in CLASS_LIST:
    y += [c]*NUM_SAMPLES
y = np.array(y)
# Read data
data = []
for c in CLASS_LIST:
    for i in range(NUM_SAMPLES):
        im_frame = Image.open(f"data_COIL20/coil-20-proc/obj{c+1}__{i}.png")
        np_frame = np.array(im_frame)
        data.append(np_frame.ravel())
    # samples per class
# going through classes
# Store data into variable
data = np.array(data)
print("All data shape")
print(data.shape)
# Read indices of subsets
indices_subsets = [] 
data_subsets = []
labels_subsets = []
for i in range(NUM_EXP):
    indices_exp = np.loadtxt(f"data_COIL20/indices_{i}.txt").astype(int).tolist()
    indices_subsets.append(indices_exp)
    data_subsets.append(data[indices_exp])
    labels_subsets.append(y[indices_exp])

In [ ]:
def draw_repr_cycle(repr_cycle, figsize, data):
    fig, ax = plt.subplots(figsize=figsize)
    G = nx.Graph()
    for v in np.unique(repr_cycle):
        G.add_node(v)
    weighted_edges = []
    for edge in np.array(repr_cycle).reshape((-1,2)).tolist():
        weighted_edges.append((edge[0], edge[1], distance.euclidean(data[edge[0]], data[edge[1]])))
    # G.add_edges_from(weighted_edges)
    G.add_weighted_edges_from(weighted_edges)
    pos = nx.spectral_layout(G)
    nx.draw_networkx(G, ax=ax, pos=pos, width=figsize[0])
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    for node in pos.keys():
        im_array = data[node].reshape((128,128))
        im_frame = Image.fromarray(im_array)
        center = origin=pos[node]
        extent = (center[0]-0.05, center[0]+0.05, center[1]-0.05, center[1]+0.05)
        ax.imshow(im_array, cmap="gray", extent=extent, zorder=4)
    
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)
    return ax

In [ ]:
output_dir="output"
indices_subset = indices_subsets[0]
ibfm_out = ibfm.get_IBloFunMatch_subset(None, data, indices_subset, output_dir, num_it=4, points=True, max_rad=-1)

In [ ]:
ibfm_out["S_barcode_1"]

In [ ]:
for repr_cycle in ibfm_out["S_reps_1"]:
    figsize = (len(repr_cycle)*0.7, len(repr_cycle)*0.7)                      
    draw_repr_cycle(repr_cycle, figsize, data)